In [2]:
import numpy as np
import tvm
from tvm.ir.module import IRModule
from tvm.script import tir as T

# tensorIR是TVM框架中使用的张量程序抽象
# 使用张量程序抽象的主要目的是表示循环和相关的硬件加速选择，如多线程、特殊硬件指令的使用和内存访问

![img](1.png)

In [3]:
# 很简单的一个实现，这里实现是利用高级API，我们并不清楚底层逻辑，下面去深究
dtype = "float32"
a_np = np.random.rand(128, 128).astype(dtype)
b_np = np.random.rand(128, 128).astype(dtype)
# a @ b is equivalent to np.matmul(a, b)
c_mm_relu = np.maximum(a_np @ b_np, 0)
print(c_mm_relu)

[[30.680641 33.87954  35.80407  ... 32.677223 33.841488 32.23138 ]
 [31.863943 33.936146 36.191948 ... 31.993778 34.667587 30.680267]
 [31.250162 34.607754 38.086098 ... 32.508224 35.365944 33.87797 ]
 ...
 [29.462214 33.859936 36.244686 ... 32.86081  33.382538 32.291607]
 [31.388216 35.82871  36.43938  ... 33.6271   32.194027 33.463955]
 [33.077515 35.386505 39.225224 ... 34.778347 34.542305 33.266495]]


In [12]:
# 稍微底层一点的实现，实际上再底层就是偏向C语言或者汇编了
def lnumpy_mm_relu(A: np.ndarray, B: np.ndarray, C: np.ndarray):
    Y = np.empty((128, 128), dtype="float32")
    for i in range(128):
        for j in range(128):
            for k in range(128):
                if k == 0:
                    Y[i, j] = 0
                Y[i, j] = Y[i, j] + A[i, k] * B[k, j]
    for i in range(128):
        for j in range(128):
            C[i, j] = max(Y[i, j], 0)
C = np.ones((128, 128))
lnumpy_mm_relu(a_np, b_np, C)
# print(C)    # 结果一致
np.testing.assert_allclose(c_mm_relu, C, rtol=1e-5)   # 看下结果是否一致

None


In [5]:
# 下面的代码块展示了 mm_relu 的 TensorIR 实现。这里的代码是用一种名为 TVMScript 的语言实现的，
# 它是一种嵌入在 Python AST 中的特定领域方言；与lnumpy_mm_relu函数对比学习

# @tvm.script.ir_module 和 @T.prim_func 这两个装饰器用于表示对应部分的类型
@tvm.script.ir_module
class MyModule:
    @T.prim_func
    # 相当于利用tvm机器学习编译对上述函数的一个改写
    def mm_relu(A: T.Buffer[(128, 128), "float32"],
                B: T.Buffer[(128, 128), "float32"],
                C: T.Buffer[(128, 128), "float32"]):
    # 参数部分，这里使用T.Buffer创建缓存相当于提前分配空间，形状为（128，128），类型是float32
    # 这些附加信息有助于可能的机器学习编译过程以生成专门针对形状和数据类型的代码
        # 这里的 global_symbol 对应函数名，tir.noalias 是一个属性，表示所有的缓冲存储器不重叠
        T.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        # TensorIR 在中间结果分配中也使用了缓冲区类型，对应lnumpy_mm_relu中的Y初始化分配
        Y = T.alloc_buffer((128, 128), dtype="float32")
        # T.grid 是 TensorIR 中的语法糖，供我们书写多个嵌套的迭代器，等价于上面的三个循环嵌套
        for i, j, k in T.grid(128, 128, 128):
            with T.block("Y"):   # 计算块， 块是TensorIR中的基本计算单位
                # 一个块包含一组块轴（vi、vj、vk）和围绕它们定义的计算
                # 附加信息（块轴范围及其属性）使块轴独立于外部循环嵌套 i, j, k
                # 块轴信息还提供了额外的属性，帮助我们验证用于执行计算的外部循环的正确性
                vi = T.axis.spatial(128, i)   # 把vi，vj，vk绑定到i,j,k上
                vj = T.axis.spatial(128, j)   # spatial为属性，vi，vj称为空间轴
                vk = T.axis.reduce(128, k)    # reduce声明vk为归约轴
                # 上述绑定过程可以简写如下
                # SSR means the properties of each axes are "spatial", "spatial", "reduce"
                # vi, vj, vk = T.axis.remap("SSR", [i, j, k])
                with T.init():
                    Y[vi, vj] = T.float32(0)
                Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
        for i, j in T.grid(128, 128):
            with T.block("C"):
                vi = T.axis.spatial(128, i)
                vj = T.axis.spatial(128, j)
                ## 同理这里也可以用remap简写
                C[vi, vj] = T.max(Y[vi, vj], T.float32(0))

In [8]:
type(MyModule["mm_relu"])

tvm.tir.function.PrimFunc